In [19]:
import pandas as pd
import os
#url_list = [url1]#
import pandas as pd
from bs4 import BeautifulSoup
import requests
def get_dribbleshots(years,ps = False):
    dribbles=['0%20Dribbles','1%20Dribble','2%20Dribbles','3-6%20Dribbles','7%2B%20Dribbles']
    terms = ['0','1','2','3_6','7+']
    folder = '/player_shooting/'
    sfolder=''
    stype = "Regular%20Season"
    if ps == True:
        stype="Playoffs"
        sfolder = "/playoffs"
    dataframe=[]
    for year in years:
        i = 0
        for dribble in dribbles:
            #print(dribble)
            season = str(year)+'-'+str(year+1 - 2000)
            part1 = "https://stats.nba.com/stats/leaguedashplayerptshot?CloseDefDistRange=&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&DribbleRange="
            part2 = "&GameScope=&GameSegment=&GeneralRange=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&Season="

            part3 = "&SeasonSegment=&SeasonType="+stype+"&ShotClockRange=&ShotDistRange=&StarterBench=&TeamID=0&TouchTimeRange=&VsConference=&VsDivision=&Weight="
            url = part1+dribble+part2+season+part3
            headers = {
                    "Host": "stats.nba.com",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                    "Accept": "application/json, text/plain, */*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip, deflate, br",

                    "Connection": "keep-alive",
                    "Referer": "https://stats.nba.com/"
                }
            json = requests.get(url,headers = headers).json()
            print(json.keys())
            data = json["resultSets"][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            json = requests.get(url,headers = headers).json()
            data = json['resultSets'][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            new_columns = {'FG2A_FREQUENCY':'2FG FREQ%',
             'FG2_PCT':'2FG%',
             'FG2A':'2FGA',
             'FG2M':'2FGM',
             'FG3A_FREQUENCY':'3FG FREQ%',
             'FG3_PCT':'3P%',
             'FG3A':'3PA',
             'FG3M':'3PM',
             'EFG_PCT':'EFG%',
             'FG_PCT':'FG%',
             'FGA_FREQUENCY':'FREQ%',
             'PLAYER_NAME':'PLAYER',
             'PLAYER_LAST_TEAM_ABBREVIATION':'TEAM'}
            df = df.rename(columns = new_columns)
            df = df [['PLAYER', 'TEAM', 'AGE', 'GP', 'G', 'FREQ%', 'FGM', 'FGA', 'FG%',
                   'EFG%', '2FG FREQ%', '2FGM', '2FGA', '2FG%', '3FG FREQ%', '3PM', '3PA',
                   '3P%']]
            for col in df.columns:
                if '%' in col or 'PERC' in col:
                    df[col]*=100
            term = terms[i]
            df['dribbles'] = term
            df['year']=year+1
            dataframe.append(df)

            i+=1
        return pd.concat(dataframe)

def master_dribble(year,ps = False):
    trail='_ps'
    if ps == False:
        trail=''
    old = pd.read_csv('dribbleshot'+trail+'.csv')
    df = get_dribbleshots([year],ps=True)
    year+=1

    old = old[old.year!=year]
    new_master = pd.concat([old,df])
    new_master.to_csv('dribbleshot'+trail+'.csv',index=False)
    return new_master
df = master_dribble(2023,ps=True)
df

dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])


,PLAYER,TEAM,AGE,GP,G,FREQ%,FGM,FGA,FG%,EFG%,2FG FREQ%,2FGM,2FGA,2FG%,3FG FREQ%,3PM,3PA,3P%,dribbles,year
0,Tim Duncan,SAS,38.0,23,23,65.4,103,172,59.9,59.9,65.0,103,171,60.2,0.4,0,1,0.0,0,2014
1,Chris Bosh,MIA,30.0,20,20,70.2,82,158,51.9,61.1,37.8,53,85,62.4,32.4,29,73,39.7,0,2014
2,Kevin Durant,OKC,25.0,19,19,38.3,81,158,51.3,62.0,18.0,47,74,63.5,20.4,34,84,40.5,0,2014
3,Paul George,IND,24.0,19,19,44.8,60,134,44.8,59.7,12.7,20,38,52.6,32.1,40,96,41.7,0,2014
4,David West,IND,33.0,19,18,54.5,75,128,58.6,59.4,50.6,73,119,61.3,3.8,2,9,22.2,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Terance Mann,LAC,27.0,4,1,3.7,0,1,0.0,0.0,3.7,0,1,0.0,0.0,0,0,NaN,7+,2024
89,Isaiah Thomas,PHX,35.0,1,1,50.0,0,1,0.0,0.0,50.0,0,1,0.0,0.0,0,0,NaN,7+,2024
90,Duncan Robinson,MIA,30.0,4,1,7.7,0,1,0.0,0.0,7.7,0,1,0.0,0.0,0,0,NaN,7+,2024
91,AJ Green,MIL,24.0,5,1,6.3,0,1,0.0,0.0,0.0,0,0,NaN,6.3,0,1,0.0,7+,2024


In [15]:
df

,PLAYER,TEAM,AGE,GP,G,FREQ%,FGM,FGA,FG%,EFG%,2FG FREQ%,2FGM,2FGA,2FG%,3FG FREQ%,3PM,3PA,3P%,dribbles,year
0,Joel Embiid,PHI,30.0,5,5,56.5,27,61,44.3,53.3,28.7,16,31,51.6,27.8,11,30,36.7,0,2024
1,Myles Turner,IND,28.0,5,5,75.9,31,60,51.7,65.8,30.4,14,24,58.3,45.6,17,36,47.2,0,2024
2,Nikola Jokic,DEN,29.0,5,5,51.6,30,48,62.5,68.8,35.5,24,33,72.7,16.1,6,15,40.0,0,2024
3,Michael Porter Jr.,DEN,25.0,5,5,63.2,26,48,54.2,72.9,15.8,8,12,66.7,47.4,18,36,50.0,0,2024
4,Anthony Davis,LAL,31.0,5,5,49.5,32,46,69.6,69.6,44.1,32,41,78.0,5.4,0,5,0.0,0,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Terance Mann,LAC,27.0,4,1,3.7,0,1,0.0,0.0,3.7,0,1,0.0,0.0,0,0,NaN,7+,2024
89,Isaiah Thomas,PHX,35.0,1,1,50.0,0,1,0.0,0.0,50.0,0,1,0.0,0.0,0,0,NaN,7+,2024
90,Duncan Robinson,MIA,30.0,4,1,7.7,0,1,0.0,0.0,7.7,0,1,0.0,0.0,0,0,NaN,7+,2024
91,AJ Green,MIL,24.0,5,1,6.3,0,1,0.0,0.0,0.0,0,0,NaN,6.3,0,1,0.0,7+,2024
